In [2]:
import PyPDF2
from pathlib import Path
import spacy
nlp =  spacy.load("en_core_web_sm")
import pdfplumber 
import re
from spellchecker import SpellChecker
from spacy.matcher import Matcher

In [12]:
def spell_checker(text):
        spell = SpellChecker()
        docs = nlp(text)
        correct_words = []    
        for token in docs:
            word = token.text

            if token.is_alpha:
                if word.lower() in spell.unknown([word]):
                    correction = spell.correction(word)
                    correct_words.append(correction)
                else:
                    correct_words.append(word)
            else:
                correct_words.append(word)
        return ' '.join(correct_words)

In [3]:
path = Path("/Users/ozgur.sahin/Documents/ragchat_local/docs/domain1/2406.16563v2.pdf")

In [29]:
print(PyPDF2.__version__)

3.0.1


In [178]:
with pdfplumber.open(path) as pdf:
    all_text = ''
    page = pdf.pages[0]
    text = page.extract_text()
    clean_text = text.replace('\n',' ')
    print(clean_text)

MEDICAL GRAPH RAG: TOWARDS SAFE MEDICAL LARGE LANGUAGE MODEL VIA GRAPH RETRIEVAL- AUGMENTED GENERATION JundeWu JiayuanZhu YunliQi UniversityofOxford UniversityofOxford UniversityofOxford jundewu@ieee.org ABSTRACT Weintroduceanovelgraph-basedRetrieval-AugmentedGeneration(RAG)frame- workspecificallydesignedforthemedicaldomain,calledMedGraphRAG,aimed atenhancingLargeLanguageModel(LLM)capabilitiesandgeneratingevidence- based results, thereby improving safety and reliability when handling private medicaldata. Ourcomprehensivepipelinebeginswithahybridstatic-semantic approach to document chunking, significantly improving context capture over traditional methods. Extracted entities are used to create a three-tier hierarchi- calgraphstructure,linkingentitiestofoundationalmedicalknowledgesourced frommedicalpapersanddictionaries. Theseentitiesaretheninterconnectedto formmeta-graphs,whicharemergedbasedonsemanticsimilaritiestodevelopa comprehensiveglobalgraph. Thisstructuresupportspreciseinformatio

In [45]:
with path.open('rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page in pdf_reader.pages:
              page_text = page.extract_text(orientations=0)
              clean_page_text = page_text.replace('\n',' ').strip()
              clean_page_text = re.sub(r'\bwww\.[^\s]*?\.(com|org|net)\b.*?\.','', clean_page_text,flags=re.IGNORECASE | re.DOTALL) # clean links 
              clean_page_text = re.sub(r'^[\d*]+[^\s]+\s+.*$','', clean_page_text,flags=re.MULTILINE)
              clean_page_text = re.sub(r'[^A-Za-z0-9\s.,!?\'"-]', '',clean_page_text) #
              clean_page_text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', clean_page_text)#edit 'T OWARDS' to 'TOWARDS'
              clean_page_text = re.sub(r'\s+([A-Z])\s+([A-Z])', r' \1\2', clean_page_text)#edit 'A TOWARDS'
              clean_page_text = re.sub(r'(\d+)\s*\n\s*([A-Za-z])', r'\1 \2', clean_page_text)
              clean_page_text = re.sub(r'^[\w\.-]+[a-zA-Z0-9\.-]+\.[a-zA-Z]{2,}$', r' ', clean_page_text) # clean emails
              clean_page_text = re.sub(r'https?[\w\.-]+\.[\w\.-]+', r' ', clean_page_text) # clean urls
              clean_page_text = re.sub(r'(?i)\btable\s+\d+[:.]*\s*[^.]*\.',r' ', clean_page_text) # clean tables
              clean_page_text = re.sub(r'\b(Fig|Figure)\b.*?\.','', clean_page_text,flags=re.IGNORECASE | re.DOTALL) #clean fig or figure texts
              clean_page_text = re.sub(r'\s\d\s','', clean_page_text) #clean ' 1 ' strings 
              clean_page_text = re.sub(r'\s+', ' ',clean_page_text)
              text += clean_page_text + ' '
        print(text)

A.7 Detailed error results N1alter and N2alter are sequence errors. 18


In [42]:
docs = nlp(text)

In [44]:
for i in docs:
    print(i)

Are
there
identifiable
structural
parts
in
the
sentence
embedding
whole
?
Vivi
Nastase1and
Paola
Merlo1,2
1Idiap
Research
Institute
,
Martigny
,
Switzerland
2University
of
Geneva
,
Swizerland
vivi.a.nastasegmail.com
,
Paola.Merlounige.ch
Abstract
Sentence
embeddings
from
transformer
models
encode
in
a
fixed
length
vector
much
linguistic
information
.
We
explore
the
hypothesis
that
these
embeddings
consist
of
overlapping
layers
of
information
that
can
be
separated
,
and
on
which
specific
types
of
information
such
as
information
about
chunks
and
their
structural
and
semantic
properties
can
be
detected
.
We
show
that
this
is
the
case
using
a
dataset
consisting
of
sentences
with
known
chunk
structure
,
and
two
linguistic
intelligence
datasets
,
solving
which
relies
on
detecting
chunks
and
their
grammatical
number
,
and
respectively
,
their
semantic
roles
,
and
through
analyses
of
the
performance
on
the
tasks
and
of
the
internal
representations
built
during
learning
.
Introduction
Transform

In [34]:
#number_pattern = [{"TEXT": {"REGEX": r"^\d."}}]
foot_note_pattern = [{"TEXT": {"REGEX": r"^[\d*]+\w.*$"}}]
figure_pattern = [{"TEXT": {"REGEX": r"(?i)\btable\s+\d+[:.]*\s*.*$"}}] #'\b(Table|TABLE)\s*\d+[\s\S]*?\.
email_pattern = [{"TEXT": {"REGEX": r"^[\w\.-]+[a-zA-Z0-9\.-]+\.[a-zA-Z]{2,}$"}}]
url_pattern = [{"TEXT": {"REGEX": r"https?[\w\.-]+\.[\w\.-]+"}}]

matcher =  Matcher(nlp.vocab)
#matcher.add("NUMBER_PATTERN", [number_pattern])
matcher.add("URL_PATTERN",[url_pattern])
matcher.add("EMAIL_PATTERN",[email_pattern])
matcher.add("FIGURE_PATTERN", [figure_pattern])

In [35]:
matches = matcher(docs)
for match_id, start, end in matches:
    span = docs[start:end]
    print(f"Matched pattern: {span.text}")
    print(start,end)
print(len(matches))

Matched pattern: vivi.a.nastasegmail.com
27 28
Matched pattern: Paola.Merlounige.ch
29 30
Matched pattern: Geneva.principal
338 339
Matched pattern: 2023.Data
1890 1891
Matched pattern: 174371404.John
5783 5784
5


In [33]:
sentences = [sent.text.strip() for sent in docs.sents]
valid_sentences = [sentences for sentence in sentences if len(sentence) > 15] 


In [158]:
dot_indexes = []
dot_pattern = [{"TEXT": "."}]
match1 = Matcher(nlp.vocab)
match1.add("DOT_PATTERN",[dot_pattern])

In [159]:
m = match1(docs)
for match_id, start, end in m:
    span = docs[start:end]
    dot_indexes.append(start)
    print(f"Matched pattern: {span.text}")
print(dot_indexes)

Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched pattern: .
Matched patt

In [132]:
for sentence in docs.sents:
    matches = matcher(sentence)
    for match_id, start, end in matches:
        span = sentence[start:end]
        print(f"Matched pattern: {span.text}")
        print(start,end)
        print(sentence)

Matched pattern: 1Idiap
2 3
Paola Merlo1,2 1Idiap Research Institute, Martigny, Switzerland 2University of Geneva, Swizerland vivi.a.nastasegmail.com,
Matched pattern: 2University
9 10
Paola Merlo1,2 1Idiap Research Institute, Martigny, Switzerland 2University of Geneva, Swizerland vivi.a.nastasegmail.com,
Matched pattern: 2020
28 29
Sentence embeddings usually fine-tuned have proven useful for a variety of high-level language processing tasks e.g. the GLUE tasks Clark et al., 2020, story continuation Ippolito et al., 2020.
Matched pattern: 2020
37 38
Sentence embeddings usually fine-tuned have proven useful for a variety of high-level language processing tasks e.g. the GLUE tasks Clark et al., 2020, story continuation Ippolito et al., 2020.
Matched pattern: 2018
44 45
Sentence embeddings built using a BiLSTM model do seem to encode a range of information, from shallow e.g. sentence length, word order to syntactic e.g. tree depth, top constituent and semantic e.g. tense, semantic misma